<a href="https://colab.research.google.com/github/robertgwx/reimagined-octo-tribble/blob/main/Opt_Daily_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Defining the function to fetch all daily data using only stations incl. in LTCE threads

In [ ]:
import pandas as pd
import seaborn as sns
from io import StringIO
import requests
import matplotlib.pyplot as plt
import calendar
import re
import os
import sys
import numpy as np
from datetime import datetime
from google.colab import drive
from tabulate import tabulate
from scipy import stats
import glob
from heapq import nlargest
import contextlib # Keep import as other functions might use it

drive.mount('/content/drive')

## DEFINE ALL FUNCTIONS
# Function to convert title case, but avoid capitalizing letters after apostrophes.
def custom_title_case(text):
    return re.sub(r"(\s|^)([a-z])|(')([a-z])",
                  lambda m: (m.group(1) or "") + (m.group(2).upper() if m.group(2) else "")
                  if m.group(1) or m.group(2) else (m.group(3) or "") + (m.group(4) if m.group(4) else ""),
                  text)

# Assuming base_url is defined globally or passed to functions that need it
base_url = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html"


def fetch_daily_data(station_id, station_name, year, current_month):
    all_daily_data = []
    current_date = datetime.now()

    # Fetch data for all months in the year
    for month in range(1, 13): # Fetch all 12 months
        # Skip months after the current month if it's the current year
        if year == current_date.year and month > current_date.month:
            break
        params = {
            "format": "csv",
            "stationID": station_id,
            "Year": year,
            "Month": month,
            "timeframe": 2,  # Daily data
        }

        try:
            response = requests.get(base_url, params=params)

            if response.status_code == 200 and response.content.strip():
                csv_data = StringIO(response.content.decode("utf-8"))
                data = pd.read_csv(csv_data)

                if not data.empty:
                    # Define desired columns
                    desired_columns = [
                        "Date/Time",
                        "Max Temp (°C)",
                        "Min Temp (°C)",
                        "Mean Temp (°C)",
                        "Total Rain (mm)",
                        "Total Snow (cm)",
                        "Total Precip (mm)",
                        "Snow on Grnd (cm)",
                        "Spd of Max Gust (km/h)"
                    ]

                    # Filter columns that actually exist in the dataframe
                    existing_columns = [col for col in desired_columns if col in data.columns]

                    # Select only existing columns
                    data = data[existing_columns]

                    # Filter data for specific month and year
                    data_month = data[
                        (pd.to_datetime(data['Date/Time']).dt.month == month) &
                        (pd.to_datetime(data['Date/Time']).dt.year == year)
                    ].copy()

                    # Format 'Date/Time' to only include date (YYYY-MM-DD)
                    data_month['Date/Time'] = pd.to_datetime(data_month['Date/Time']).dt.strftime('%Y-%m-%d')

                    # Add station info to daily data
                    data_month['Station Name'] = station_name
                    data_month['Station ID'] = station_id
                    all_daily_data.extend(data_month.to_dict('records'))

        except Exception as e:
            # Keep original print for now or modify to accept stream if needed elsewhere
            print(f"Error fetching data for {station_name} in {year}, month {month}: {e}")


    return all_daily_data

def fetch_daily_data_range(station_id, station_name, start_date, end_date, output_stream=sys.stdout):
    """
    Fetch daily weather data for a specific station within a given date range.

    Args:
        station_id (str): Unique identifier for the weather station
        station_name (str): Name of the weather station
        start_date (datetime): Starting date for data retrieval
        end_date (datetime): Ending date for data retrieval
        output_stream: Stream to write output messages (e.g., sys.stdout or widget output)

    Returns:
        list: Daily weather records within the specified date range
    """
    # Validate date range
    if start_date > end_date:
        print("Error: Start date must be before or equal to end date", file=output_stream)
        return []

    # Calculate years involved in the date range
    years_involved = sorted(list(set(range(start_date.year, end_date.year + 1))))

    daily_records = []

    for year in years_involved:
        # Determine start and end months for the current year
        start_month = start_date.month if year == start_date.year else 1
        end_month = end_date.month if year == end_date.year else 12

        for month in range(start_month, end_month + 1):
            # If it's the end year and the current month is after the end_date's month, stop.
            if year == end_date.year and month > end_date.month:
                break

            # Fetch daily data for the specific month and year
            params = {
                "format": "csv",
                "stationID": station_id,
                "Year": year,
                "Month": month,
                "timeframe": 2,  # Daily data
            }

            try:
                response = requests.get(base_url, params=params)

                if response.status_code == 200 and response.content.strip():
                    csv_data = StringIO(response.content.decode("utf-8"))
                    data = pd.read_csv(csv_data)

                    if not data.empty:
                        # Define desired columns
                        desired_columns = [
                            "Date/Time",
                            "Max Temp (°C)",
                            "Min Temp (°C)",
                            "Mean Temp (°C)",
                            "Total Rain (mm)",
                            "Total Snow (cm)",
                            "Total Precip (mm)",
                            "Snow on Grnd (cm)",
                            "Spd of Max Gust (km/h)"
                        ]

                        # Filter columns that actually exist in the dataframe
                        existing_columns = [col for col in desired_columns if col in data.columns]

                        # Select only existing columns
                        data = data[existing_columns]

                        # Filter data for the specific date range within this month
                        data['Date/Time'] = pd.to_datetime(data['Date/Time'])
                        data_range_month = data[
                            (data['Date/Time'].dt.date >= start_date.date()) &
                            (data['Date/Time'].dt.date <= end_date.date())
                        ].copy()


                        if not data_range_month.empty:
                            # Format 'Date/Time' to only include date (YYYY-MM-DD)
                            data_range_month['Date/Time'] = data_range_month['Date/Time'].dt.strftime('%Y-%m-%d')

                            # Add station info to daily data
                            data_range_month['Station Name'] = station_name
                            data_range_month['Station ID'] = station_id
                            daily_records.extend(data_range_month.to_dict('records'))

            except Exception as e:
                print(f"Error fetching data for {station_name} in {year}, month {month}: {e}", file=output_stream)

    return daily_records


def get_locations_from_csv(folder_path):
    """Gets a list of unique locations from CSV filenames in a folder.

    Args:
        folder_path: The path to the folder containing the CSV files.

    Returns:
        A list of unique location names.
    """
    locations = set()  # Use a set to avoid duplicates
    if not os.path.exists(folder_path):
        return []
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # Extract location name using regular expression
            match = re.search(r"(.+)_daily_data\.csv", filename)
            if match:
                location_name = match.group(1)
                locations.add(location_name)
    return list(locations)

def remove_duplicate_dates(filepath, output_stream=sys.stdout):
    """
    Removes duplicate dates from a single CSV file, keeping the row with more data.
    """
    columns_to_check = ['Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)', 'Total Precip (mm)',
                        'Total Rain (mm)', 'Total Snow (cm)', 'Snow on Grnd (cm)', 'Spd of Max Gust (km/h)']
    try:
        df = pd.read_csv(filepath)
        df['Date/Time'] = pd.to_datetime(df['Date/Time'])
        df = df.sort_values(by=['Date/Time'])

        # Group by date and select the best row in each group
        # Using .agg to specify columns to check for max non-null count
        agg_dict = {col: 'first' for col in df.columns}
        # Use a lambda that finds the index of the row with the most non-null values for the specified columns
        for col in columns_to_check:
            if col in df.columns:
                 agg_dict[col] = lambda x: x.loc[x.notnull().sum().idxmax()] if x.notnull().sum() > 0 else np.nan


        # Ensure all columns are included, using first for those not in columns_to_check
        for col in df.columns:
            if col not in agg_dict:
                agg_dict[col] = 'first'

        # Apply aggregation and reset index
        df = df.groupby('Date/Time').agg(agg_dict).reset_index()


        df.to_csv(filepath, index=False)
        print(f"Duplicate data removal complete for {os.path.basename(filepath)}.", file=output_stream)

    except Exception as e:
        print(f"Error processing {filepath}: {e}", file=output_stream)

# New function to update a single CSV file with the most recent data
def update_csv_file(file_path):
    """
    Updates a single CSV file with the most recent data.
    Prints status messages to standard output (will be captured by Output widget).
    """
    try:
        print(f"\nUpdating file: {file_path}")

        # Read existing data
        existing_df = pd.read_csv(file_path)

        if existing_df.empty:
            print(f"File {file_path} is empty. Skipping.")
            return

        # Get the most recent Station ID and name from existing data
        # Use .iloc[0] in case the dataframe is sorted descending by date
        most_recent_record = existing_df.sort_values(by='Date/Time', ascending=False).iloc[0]
        most_recent_station_id = most_recent_record['Station ID']
        station_name = most_recent_record['Station Name']


        # Get the most recent date from the CSV
        most_recent_date_str = most_recent_record['Date/Time']
        most_recent_date = pd.to_datetime(most_recent_date_str)


        # Get current date
        current_date = datetime.now()

        print(f"  Most recent data is from: {most_recent_date.strftime('%Y-%m-%d')}")
        print(f"  Current date: {current_date.strftime('%Y-%m-%d')}")

        # Determine the date from which to start fetching new data
        # Start fetching from the day after the most recent date
        start_fetch_date = most_recent_date + pd.Timedelta(days=1)

        # If the start fetch date is today or in the future, no new data is needed
        if start_fetch_date.date() >= current_date.date():
            print(f"  Data is already up to date (last record from {most_recent_date.strftime('%Y-%m-%d')}). No update needed.")
            return

        print(f"  Fetching new data from {start_fetch_date.strftime('%Y-%m-%d')} to {current_date.strftime('%Y-%m-%d')}...")

        # Fetch new data for the required date range, passing the output stream
        all_new_records = fetch_daily_data_range(most_recent_station_id, station_name, start_fetch_date, current_date)


        if all_new_records:
            # Convert new records to DataFrame
            new_df = pd.DataFrame(all_new_records)

            # Define columns to check for non-null values
            columns_to_check = ['Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)', 'Total Precip (mm)',
                                'Total Rain (mm)', 'Total Snow (cm)', 'Snow on Grnd (cm)', 'Spd of Max Gust (km/h)']

            # Remove rows where ALL specified columns are empty/NaN in the new data
            new_df = new_df.dropna(subset=columns_to_check, how='all')

            if not new_df.empty:
                # Ensure 'Date/Time' is datetime for comparison
                existing_df['Date/Time'] = pd.to_datetime(existing_df['Date/Time'])
                new_df['Date/Time'] = pd.to_datetime(new_df['Date/Time'])

                # Filter out dates from new_df that are already in existing_df
                new_df_filtered = new_df[~new_df['Date/Time'].isin(existing_df['Date/Time'])].copy()

                if not new_df_filtered.empty:
                    # Format 'Date/Time' back to string for saving
                    new_df_filtered['Date/Time'] = new_df_filtered['Date/Time'].dt.strftime('%Y-%m-%d')
                    existing_df['Date/Time'] = existing_df['Date/Time'].dt.strftime('%Y-%m-%d')

                    # Combine existing data with new data
                    df_comprehensive_data = pd.concat([existing_df, new_df_filtered], ignore_index=True)

                    # Save the data to the CSV file
                    df_comprehensive_data.to_csv(file_path, mode="w", index=False)

                    print(f"  Updated with {len(new_df_filtered)} new records.")

                else:
                    print(f"  No new data found after filtering existing dates.")
            else:
                 print(f"  No new data found with valid weather data.")
        else:
            print(f"  No new data fetched from the source.")

    except Exception as e:
        print(f"  Error updating {file_path}: {str(e)}")

# Main script logic (This is likely not used by the widget directly, but keeping it for completeness)
def main():
    # Ask the user what they want to do
    print("\nClimate Data Update Options:")
    print("1. Update a specific location's data")
    print("2. Update all CSV files with the most recent data")

    choice = input("\nEnter your choice (1 or 2): ").strip()

    if choice == "1":
        # Original code for updating a specific location
        # Get the province abbreviation from the user
        while True:
            province = input("Enter the province abbreviation (e.g., NL, NS): ").upper()
            if province in valid_provinces:
                break  # Exit the loop if the input is valid
            else:
                print("Invalid province abbreviation. Please enter a valid abbreviation.")

        # Get the location name from the user
        location_name = input("Enter the location name to search for: ")
        location_name = custom_title_case(location_name)

        # Rest of your original code for specific location update
        try:
            # Load climate stations
            climate_stations = pd.read_csv('/content/drive/MyDrive/Climate Data/Station Inventory EN.csv')

            # Find matching station IDs
            location_name_modified = location_name.replace(".", "")  # Remove periods from input
            matching_stations = climate_stations[
                matching_stations['Name'].str.replace(".", "", regex=False).str.lower().str.startswith(location_name_modified.lower(), na=False)
            ]

            if matching_stations.empty:
                print(f"No stations found matching the location name: {location_name}")
                return

            station_info = {}
            for index, row in matching_stations.iterrows():
                station_id = row['Station ID']
                station_name = custom_title_case(location_name)
                first_year = row['First Year']
                last_year = row['Last Year']
                station_info[station_id] = {'name': station_name, 'years': range(first_year, last_year + 1)}

            # Get current year and month
            current_year = datetime.now().year
            current_month = datetime.now().month

            # Create the output filename for daily data
            output_file_daily = f"/content/drive/MyDrive/Climate Data/Daily Data/{province}/{station_name}_daily_data.csv"

            # Check if the file exists
            all_daily_records = []

            if os.path.exists(output_file_daily):
                # Update existing file with new data for current month
                update_csv_file(output_file_daily)
            else:
                # If file doesn't exist, collect data for all years
                for station_id, info in station_info.items():
                    station_name = info['name']
                    years = list(info['years'])  # Convert to list to get total years
                    print(f"\nFetching data for station: {station_name} ({station_id})")
                    print(f"Total years to fetch: {len(years)}")

                    # Fetch data for all years
                    for year_index, year in enumerate(years, 1):
                        print(f"  Fetching data for Year {year} ({year_index}/{len(years)}) ...")
                        daily_records = fetch_daily_data(station_id, station_name, year, current_month)
                        all_daily_records.extend(daily_records)

                # Create DataFrame for daily records
                df_comprehensive_data = pd.DataFrame(all_daily_records)

                # Remove rows where ALL specified columns are empty/NaN
                df_comprehensive_data = df_comprehensive_data.dropna(subset=columns_to_check, how='all')

                # Create directory if it doesn't exist
                os.makedirs(os.path.dirname(output_file_daily), exist_ok=True)

                # Save the data to a CSV file
                df_comprehensive_data.to_csv(output_file_daily, mode="w", index=False)

                print(f"\nFull data collection completed. Saved to {output_file_daily}")

                # Remove duplicate dates
                remove_duplicate_dates(output_file_daily)

        except FileNotFoundError:
            print("Error: Station Inventory EN.csv not found. Please upload the file.")

    elif choice == "2":
        # New code for updating all CSV files
        base_dir = "/content/drive/MyDrive/Climate Data/Daily Data"

        if not os.path.exists(base_dir):
            print(f"Error: Directory {base_dir} not found.")
            return

        # Find all CSV files in all province subdirectories
        all_csv_files = []

        for province in valid_provinces:
            province_dir = os.path.join(base_dir, province)
            if os.path.exists(province_dir):
                csv_pattern = os.path.join(province_dir, "*.csv")
                province_csv_files = glob.glob(csv_pattern)
                all_csv_files.extend(province_csv_files)

        if not all_csv_files:
            print("No CSV files found in any province directories.")
            return

        print(f"\nFound {len(all_csv_files)} CSV files to update.")

        # Ask for confirmation
        confirm = input(f"Do you want to update all {len(all_csv_files)} files? (y/n): ").strip().lower()

        if confirm == 'y':
            # Update each CSV file
            for i, file_path in enumerate(all_csv_files, 1):
                print(f"\nUpdating file {i}/{len(all_csv_files)}: {os.path.basename(file_path)}")
                update_csv_file(file_path)

            print("\nAll files have been updated with the most recent data.")
        else:
            print("Update cancelled.")

    else:
        print("Invalid choice. Please run the script again and select option 1 or 2.")

# Defining the lookup function for data searches

In [ ]:
def get_meteorological_season(month):
    """Helper function to determine meteorological season from month"""
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    else:  # 6, 7, 8, 9, 10, 11
        return 'Summer' if month in [6, 7, 8] else 'Fall'

def lookup_data(df, column, lookup_type, value=None, location=None, year=None, month=None):
    """
    Perform flexible data lookups in a weather dataset

    Parameters:
    -----------
    df : pandas.DataFrame
        The comprehensive weather dataset
    column : str
        The column to search (e.g., 'Max Temp (°C)', 'Total Precip (mm)')
    lookup_type : str
        Type of lookup to perform. Options:
        - 'last_occurrence': Find the last time a specific value occurred
        - 'first_occurrence': Find the first time a specific value occurred
        - 'last_greater_than_or_equal': Last occurrence where value is >= specified value
        - 'last_less_than_or_equal': Last occurrence where value is <= specified value
        - 'count_greater_than_or_equal': Count of occurrences where value is >= specified value
        - 'count_less_than_or_equal': Count of occurrences where value is <= specified value
        - 'max_in_year': Maximum value for a given year
        - 'min_in_year': Minimum value for a given year
        - 'max_in_month': Maximum value for a given month
        - 'min_in_month': Minimum value for a given month
        - 'overall_max': Maximum value in entire dataset
        - 'overall_min': Minimum value in entire dataset
        - 'all_occurrences': Find all occurrences of a specific value
        - 'all_greater_than_or_equal': Find all occurrences where value is >= specified value
        - 'all_less_than_or_equal': Find all occurrences where value is <= specified value


    value : float, optional
        The specific value to look for (used with several lookup types)
    location : str, optional
        The specific location/station name to filter by
    year : int, optional
        The year to filter by
    month : int, optional
        The month to filter by

    Returns:
    --------
    result : pandas.DataFrame or float or int
        Depending on the lookup type, returns various types of results. For lookup types that return multiple occurrences, a DataFrame is returned.
    """
    # Validate column exists
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in the dataset")

    # Create a copy of the dataframe to avoid modifying the original
    data = df.copy()

    # Apply location filtering if specified
    if location is not None:
        # Use case-insensitive matching for location
        data = data[data['Station Name'].str.lower() == location.lower()]

        if data.empty:
            # Return empty DataFrame for consistency if no data for location
            return pd.DataFrame()

    # Convert the column to numeric, coercing errors to NaN
    data[column] = pd.to_numeric(data[column], errors='coerce')

    # Apply filtering based on year and month if provided
    if year is not None:
        data = data[data['Year'] == year]

    if month is not None:
        data = data[data['Month'] == month]


    # --- Lookup Types Returning DataFrames (Multiple Occurrences) ---
    if lookup_type in ['last_occurrence', 'first_occurrence',
                       'last_greater_than_or_equal', 'last_less_than_or_equal',
                       'all_occurrences', 'all_greater_than_or_equal', 'all_less_than_or_equal']:
        if value is None and lookup_type in ['last_occurrence', 'first_occurrence',
                                             'last_greater_than_or_equal', 'last_less_than_or_equal',
                                             'all_occurrences', 'all_greater_than_or_equal', 'all_less_than_or_equal']:
             raise ValueError("'value' must be specified for this lookup type")

        if lookup_type in ['last_occurrence', 'first_occurrence', 'all_occurrences']:
             matches = data[data[column] == value].copy()
        elif lookup_type in ['last_greater_than_or_equal', 'all_greater_than_or_equal']:
             matches = data[data[column] >= value].copy()
        elif lookup_type in ['last_less_than_or_equal', 'all_less_than_or_equal']:
             matches = data[data[column] <= value].copy()

        if matches.empty:
            return pd.DataFrame() # Return empty DataFrame if no matches

        # Sort by date/time for consistent ordering
        matches = matches.sort_values(by='Date/Time')

        # For 'last' lookups, filter to only the last occurrence
        if lookup_type in ['last_occurrence', 'last_greater_than_or_equal', 'last_less_than_or_equal']:
             return matches.tail(1) # Return DataFrame with the last row
        elif lookup_type in ['first_occurrence']:
             return matches.head(1) # Return DataFrame with the first row
        else: # 'all_occurrences', 'all_greater_than_or_equal', 'all_less_than_or_equal'
             return matches # Return DataFrame with all matches


    # --- Lookup Types Returning Single Values or Counts ---
    elif lookup_type == 'count_greater_than_or_equal':
        if value is None:
            raise ValueError("'value' must be specified for count_greater_than_or_equal lookup")
        # Count occurrences where value is greater than or equal to specified value
        count = (data[column] >= value).sum()
        return count

    elif lookup_type == 'count_less_than_or_equal':
        if value is None:
            raise ValueError("'value' must be specified for count_less_than_or_equal lookup")
        # Count occurrences where value is less than or equal to specified value
        count = (data[column] <= value).sum()
        return count

    elif lookup_type in ['max_in_year', 'min_in_year', 'max_in_month', 'min_in_month', 'overall_max', 'overall_min']:
        # Convert the column to numeric, coercing errors to NaN
        data[column] = pd.to_numeric(data[column], errors='coerce')

        if lookup_type == 'max_in_year':
            if year is None:
                raise ValueError("'year' must be specified for max_in_year lookup")
            max_value = data[column].max()
            if pd.isna(max_value): return None, None # Handle case with no data
            # Return DataFrame with all rows matching the max value
            max_rows = data[data[column] == max_value].copy()
            return max_rows # Return DataFrame

        elif lookup_type == 'min_in_year':
            if year is None:
                raise ValueError("'year' must be specified for min_in_year lookup")
            min_value = data[column].min()
            if pd.isna(min_value): return None, None # Handle case with no data
            # Return DataFrame with all rows matching the min value
            min_rows = data[data[column] == min_value].copy()
            return min_rows # Return DataFrame

        elif lookup_type == 'max_in_month':
            if month is None:
                raise ValueError("'month' must be specified for max_in_month lookup")
            max_value = data[column].max()
            if pd.isna(max_value): return None, None # Handle case with no data
            # Return DataFrame with all rows matching the max value
            max_rows = data[data[column] == max_value].copy()
            return max_rows # Return DataFrame

        elif lookup_type == 'min_in_month':
            if month is None:
                raise ValueError("'month' must be specified for min_in_month lookup")
            min_value = data[column].min()
            if pd.isna(min_value): return None, None # Handle case with no data
            # Return DataFrame with all rows matching the min value
            min_rows = data[data[column] == min_value].copy()
            return min_rows # Return DataFrame


        elif lookup_type == 'overall_max':
            # Remove any filtering by year or month for overall lookups
            data_overall = df.copy()
            if location is not None:
                data_overall = data_overall[data_overall['Station Name'].str.lower() == location.lower()]
            # Convert the column to numeric, coercing errors to NaN
            data_overall[column] = pd.to_numeric(data_overall[column], errors='coerce')
            max_value = data_overall[column].max()
            if pd.isna(max_value): return pd.DataFrame() # Handle case with no data
            # Return DataFrame with all rows matching the overall max value
            max_rows = data_overall[data_overall[column] == max_value].copy()
            return max_rows # Return DataFrame

        elif lookup_type == 'overall_min':
            # Remove any filtering by year or month for overall lookups
            data_overall = df.copy()
            if location is not None:
                data_overall = data_overall[data_overall['Station Name'].str.lower() == location.lower()]
            # Convert the column to numeric, coercing errors to NaN
            data_overall[column] = pd.to_numeric(data_overall[column], errors='coerce')
            min_value = data_overall[column].min()
            if pd.isna(min_value): return pd.DataFrame() # Handle case with no data
            # Return DataFrame with all rows matching the overall min value
            min_rows = data_overall[data_overall[column] == min_value].copy()
            return min_rows # Return DataFrame

    else:
        raise ValueError(f"Invalid lookup_type: {lookup_type}")

# Widget Code

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
from heapq import nlargest
import calendar
import os

class InteractiveWeatherAnalyzer:
    def __init__(self, base_folder_path):
        self.base_folder_path = base_folder_path
        self.valid_provinces = ["NB", "NL", "NS", "PEI", "QC"]
        self.df = None
        self.locations = []
        self.current_province = None
        self.analysis_results = [] # Store structured results (list of dicts)
        self.current_result_index = 0 # Track current position in results

        # Create all widgets
        self.setup_widgets()
        self.setup_layout()

    def setup_widgets(self):
        # === SELECTION WIDGETS ===
        self.province_selector = widgets.Dropdown(
            options=self.valid_provinces,
            value='NL',
            description='Province:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='200px')
        )

        self.location_selector = widgets.Dropdown(
            options=[],
            description='Location:',
            disabled=True,
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )

        # === OPTIONS ===
        self.show_details_toggle = widgets.Checkbox(
            value=False,
            description='Show details',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px')
        )

        self.auto_refresh_toggle = widgets.Checkbox(
            value=True,
            description='Auto-refresh',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px')
        )

        # === YEAR RANGE SELECTOR CHECKBOX ===
        self.use_year_range_toggle = widgets.Checkbox(
            value=False,
            description='Use year range',
            disabled=True,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'),
            tooltip='Check to use start/end years, uncheck for single year'
        )

        # === ANALYSIS TYPE SELECTOR WITH DEFAULT ===
        self.analysis_selector = widgets.Dropdown(
            options=[
                ('Last occurrence of value', 'last_occurrence'),
                ('Max/Min in specific year', 'max_min_year'),
                ('Max/Min in specific month', 'max_min_month'),
                ('Overall max/min', 'overall_max_min'),
                ('Last >= value', 'last_gte'),
                ('Last <= value', 'last_lte'),
                ('Count >= value', 'count_gte'),
                ('Count <= value', 'count_lte'),
                ('Wind gust frequencies', 'wind_freq'),
                ('Seasonal precipitation', 'seasonal_precip'),
                ('Monthly precipitation plot', 'plot_monthly'),
                ('List occurrences above threshold', 'list_threshold'),
                ('Top 5 winter snowfall', 'winter_snow')
            ],
            value='last_occurrence',
            description='Analysis:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='400px')
        )

        # === COLUMN SELECTOR ===
        self.column_selector = widgets.Dropdown(
            options=[],
            description='Column:',
            disabled=True,
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )

        # === PARAMETER WIDGETS ===
        self.value_input = widgets.FloatText(
            description='Value:',
            disabled=False,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='180px')
        )

        self.year_input = widgets.IntText(
            description='Year:',
            value=2024,
            disabled=True,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='180px'),
            tooltip='Single year for analysis'
        )

        self.month_selector = widgets.Dropdown(
            options=[(calendar.month_name[i], i) for i in range(1, 13)],
            description='Month:',
            disabled=True,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='200px')
        )

        self.start_year_input = widgets.IntText(
            description='Start:',
            value=1990,
            disabled=True,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='150px'),
            tooltip='Start year for range analysis'
        )

        self.end_year_input = widgets.IntText(
            description='End:',
            value=2025,
            disabled=True,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='150px'),
            tooltip='End year for range analysis'
        )

        self.max_min_selector = widgets.ToggleButtons(
            options=['Max', 'Min'],
            description='Type:',
            disabled=True,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='200px')
        )

        self.precip_type_selector = widgets.Dropdown(
            options=[
                ('Rain only', 'rain'),
                ('Snow only', 'snow'),
                ('Total precipitation', 'precip'),
                ('Rain + Snow combined', 'rain_snow')
            ],
            description='Type:',
            disabled=True,
            style={'description_width': '60px'},
            layout=widgets.Layout(width='250px')
        )

        # === ACTION BUTTONS ===
        self.load_button = widgets.Button(
            description='Load Data',
            button_style='primary',
            layout=widgets.Layout(width='150px', height='35px'),
            tooltip='Load climate data for selected location'
        )

        self.analyze_button = widgets.Button(
            description='Analyze',
            button_style='success',
            disabled=True,
            layout=widgets.Layout(width='150px', height='35px'),
            tooltip='Run the selected analysis'
        )

        self.clear_button = widgets.Button(
            description='Clear',
            button_style='warning',
            layout=widgets.Layout(width='100px', height='35px'),
            tooltip='Clear results'
        )

        # === Update Widgets ===
        self.update_type_selector = widgets.RadioButtons(
            options=[('Update Selected Location', 'single'), ('Update All Locations', 'all')],
            value='single',
            description='Update:',
            disabled=True,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='auto')
        )

        self.run_update_button = widgets.Button(
            description='Run Update',
            button_style='info',
            disabled=True,
            layout=widgets.Layout(width='150px', height='35px'),
            tooltip='Fetch and update data based on selection'
        )

        # === Navigation Widgets for Multiple Results ===
        self.prev_result_button = widgets.Button(
            description='Previous',
            disabled=True,
            layout=widgets.Layout(width='100px')
        )
        self.next_result_button = widgets.Button(
            description='Next',
            disabled=True,
            layout=widgets.Layout(width='100px')
        )
        self.result_counter_label = widgets.Label(value="Result 0/0")


        # === PROGRESS BAR ===
        self.progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Loading:',
            style={'description_width': '60px'},
            layout=widgets.Layout(width='300px', display='none')
        )

        # === OUTPUT AREA ===
        self.output_area = widgets.Output(
            layout=widgets.Layout(
                width='auto',
                height='auto',
                min_height='100px',
                max_height='600px',
                border='1px solid var(--jp-border-color1, #ccc)',
                overflow='auto',
                padding='10px'
            )
        )

        # === RESULTS AND PLOTS PANEL (OUTER CONTAINER) ===
        self.results_plots_panel = widgets.VBox([
            widgets.HTML("<h3>📋 Results & Visualizations</h3>"),
            widgets.HBox([self.prev_result_button, self.result_counter_label, self.next_result_button]), # Add navigation buttons
            self.output_area
        ], layout=widgets.Layout(
            border='1px solid var(--jp-border-color1, #ddd)',
            padding='10px',
            margin='5px',
            width='auto' # Initialize with auto
        ))

        # Setup event handlers
        self.setup_event_handlers()

    def setup_event_handlers(self):
        # Main event handlers
        self.province_selector.observe(self.on_province_change, names='value')
        self.location_selector.observe(self.on_location_change, names='value')
        self.analysis_selector.observe(self.on_analysis_change, names='value')
        self.column_selector.observe(self.on_column_change, names='value')

        # Year range toggle handler
        self.use_year_range_toggle.observe(self.on_year_range_toggle, names='value')

        # Button handlers
        self.load_button.on_click(self.load_data)
        self.analyze_button.on_click(self.run_analysis)
        self.clear_button.on_click(self.clear_output)
        self.run_update_button.on_click(self.run_data_update) # Event handler for the new update button
        self.update_type_selector.observe(self.on_update_type_change, names='value') # Handler for update type selection

        # Navigation button handlers
        self.prev_result_button.on_click(self.display_previous_result)
        self.next_result_button.on_click(self.display_next_result)


        # Auto-refresh handlers (if enabled)
        self.value_input.observe(self.auto_analyze, names='value')
        self.year_input.observe(self.auto_analyze, names='value')
        self.month_selector.observe(self.auto_analyze, names='value')
        self.use_year_range_toggle.observe(self.auto_analyze, names='value')

    def resize_output_area(self, content_type='simple'):
        """Dynamically resize output area based on content type and call results panel resize"""
        if content_type == 'simple':
            # For simple results (single values, short text)
            self.output_area.layout.width = 'auto'
            self.output_area.layout.min_width = '400px'
            self.output_area.layout.max_width = '600px'
            self.output_area.layout.height = 'auto'
            self.output_area.layout.min_height = '100px'
            self.output_area.layout.max_height = '300px'
            self.resize_results_panel('simple')
        elif content_type == 'table':
            # For tables and lists
            self.output_area.layout.width = 'auto'
            self.output_area.layout.min_width = '600px'
            self.output_area.layout.max_width = '900px'
            self.output_area.layout.height = 'auto'
            self.output_area.layout.min_height = '200px'
            self.output_area.layout.max_height = '500px'
            self.resize_results_panel('table')
        elif content_type == 'wide':
            # For wide content like detailed statistics
            self.output_area.layout.width = '100%'
            self.output_area.layout.height = 'auto'
            self.output_area.layout.min_height = '200px'
            self.output_area.layout.max_height = '600px'
            self.resize_results_panel('wide')
        elif content_type == 'loading':
            # For loading messages
            self.output_area.layout.width = 'auto'
            self.output_area.layout.min_width = '300px'
            self.output_area.layout.max_width = '500px'
            self.output_area.layout.height = 'auto'
            self.output_area.layout.min_height = '80px'
            self.output_area.layout.max_height = '150px'
            self.resize_results_panel('simple') # Use simple size for loading
        elif content_type == 'plot':
            # For plots
            self.output_area.layout.width = '100%'
            self.output_area.layout.height = 'auto'
            self.output_area.layout.min_height = '300px'
            self.output_area.layout.max_height = '600px'
            self.resize_results_panel('plot')

    def resize_results_panel(self, content_type='simple'):
        """Dynamically resize the outer results panel border based on content type"""
        if content_type == 'simple':
            self.results_plots_panel.layout.width = 'auto'
            self.results_plots_panel.layout.min_width = '450px'
            self.results_plots_panel.layout.max_width = '650px'
            self.results_plots_panel.layout.flex = '0 0 auto' # Don't grow/shrink, just take required width
        elif content_type == 'table':
            self.results_plots_panel.layout.width = 'auto'
            self.results_plots_panel.layout.min_width = '650px'
            self.results_plots_panel.layout.max_width = '950px'
            self.results_plots_panel.layout.flex = '0 0 auto'
        elif content_type == 'wide':
            self.results_plots_panel.layout.width = '100%'
            self.results_plots_panel.layout.flex = '1 1 auto' # Allow to grow and take available space
        elif content_type == 'plot':
             self.results_plots_panel.layout.width = '100%'
             self.results_plots_panel.layout.flex = '1 1 auto'


    def show_plot_area(self, show=True):
        """This method is no longer needed as plot is in the same area"""
        pass

    def on_year_range_toggle(self, change):
        """Handle year range checkbox toggle"""
        use_range = change['new']
        analysis = self.analysis_selector.value

        # Only affect count analyses
        if analysis in ['count_gte', 'count_lte']:
            if use_range:
                # Enable range inputs, disable single year
                self.start_year_input.disabled = False
                self.end_year_input.disabled = False
                self.year_input.disabled = True
            else:
                # Enable single year, disable range inputs
                self.year_input.disabled = False
                self.start_year_input.disabled = True
                self.end_year_input.disabled = True

    def on_province_change(self, change):
        province = change['new']
        self.current_province = province
        folder_path = f"{self.base_folder_path}/{province}"

        try:
            self.locations = get_locations_from_csv(folder_path)
            location_options = [('All Locations', 'ALL')] + [(loc, loc) for loc in sorted(self.locations)]
            self.location_selector.options = location_options
            self.location_selector.disabled = False
            self.location_selector.value = None
            self.update_type_selector.disabled = False # Enable update options when province changes
            self.on_update_type_change({'new': self.update_type_selector.value}) # Trigger update button state based on default selection

        except Exception as e:
            self.location_selector.options = []
            self.location_selector.disabled = True
            self.update_type_selector.disabled = True
            self.run_update_button.disabled = True
            self.resize_output_area('simple')
            with self.output_area:
                clear_output()
                print(f"Error loading locations: {str(e)}")

    def on_location_change(self, change):
        if change['new'] is not None:
            self.load_button.disabled = False
            # Re-evaluate update button state based on new location selection
            self.on_update_type_change({'new': self.update_type_selector.value})


    def on_update_type_change(self, change):
        """Handle change in update type selector (single vs all)"""
        update_type = change['new']
        location = self.location_selector.value

        if update_type == 'single':
            # Enable update button only if a specific location is selected
            self.run_update_button.disabled = (location == 'ALL' or location is None)
        elif update_type == 'all':
            # Enable update button if province is selected and there are locations
            self.run_update_button.disabled = (self.current_province is None or not self.locations)


    def on_analysis_change(self, change):
        analysis = change['new']

        # Reset all parameter widgets
        param_widgets = [
            self.value_input, self.year_input, self.month_selector,
            self.start_year_input, self.end_year_input, self.max_min_selector,
            self.precip_type_selector, self.use_year_range_toggle
        ]

        for widget in param_widgets:
            widget.disabled = True

        # Enable relevant widgets based on analysis type
        if analysis in ['last_occurrence', 'last_gte', 'last_lte', 'list_threshold', 'overall_max_min', 'max_min_year', 'min_in_year', 'max_in_month', 'min_in_month']: # Added overall_max_min, max_min_year, etc. here
            self.value_input.disabled = False # Value input can be used for filtering even if not required by lookup_data

        if analysis in ['count_gte', 'count_lte']:
            self.value_input.disabled = False
            self.use_year_range_toggle.disabled = False

            # Set initial state based on checkbox
            if self.use_year_range_toggle.value:
                self.start_year_input.disabled = False
                self.end_year_input.disabled = False
            else:
                self.year_input.disabled = False

        if analysis in ['max_min_year', 'seasonal_precip', 'winter_snow']:
            self.year_input.disabled = False

        if analysis in ['max_min_year', 'max_min_month', 'overall_max_min']:
            self.max_min_selector.disabled = False

        if analysis == 'max_min_month':
            self.month_selector.disabled = False

        if analysis in ['wind_freq', 'list_threshold', 'plot_monthly']: # Added list_threshold and plot_monthly here
            self.start_year_input.disabled = False
            self.end_year_input.disabled = False

        if analysis == 'plot_monthly':
            self.month_selector.disabled = False
            self.precip_type_selector.disabled = False

        # Update column options based on analysis
        self.update_column_options(analysis)

        # Reset navigation state when analysis type changes
        self.analysis_results = [] # Reset to empty list
        self.current_result_index = 0
        self.update_navigation_buttons()


    def update_column_options(self, analysis):
        if self.df is None:
            return

        if analysis in ['wind_freq']:
            # Auto-select wind gust column
            self.column_selector.options = [("Wind Gust Speed", "Spd of Max Gust (km/h)")]
            self.column_selector.value = "Spd of Max Gust (km/h)"
            self.column_selector.disabled = True

        elif analysis in ['seasonal_precip', 'plot_monthly']:
            # Precipitation columns only
            precip_cols = [col for col in self.df.columns
                          if any(x in col for x in ['Precip', 'Snow', 'Rain'])]
            self.column_selector.options = [(col, col) for col in precip_cols]
            self.column_selector.disabled = False

        elif analysis == 'winter_snow':
            # Auto-select snow column
            self.column_selector.options = [("Total Snow", "Total Snow (cm)")]
            self.column_selector.value = "Total Snow (cm)"
            self.column_selector.disabled = True

        else:
            # All numeric weather columns
            weather_cols = [col for col in self.df.columns
                           if any(x in col for x in ['Temp', 'Precip', 'Snow', 'Rain', 'Gust'])]
            self.column_selector.options = [(col, col) for col in weather_cols]
            self.column_selector.disabled = False


    def on_column_change(self, change):
        if self.auto_refresh_toggle.value and change['new'] is not None:
            self.auto_analyze()

    def auto_analyze(self, change=None):
        if (self.auto_refresh_toggle.value and
            self.df is not None and
            not self.analyze_button.disabled):
            self.run_analysis(None)

    def load_data(self, button):
        self.resize_output_area('loading')
        self.progress_bar.layout.display = 'block'
        self.progress_bar.value = 20

        with self.output_area:
            clear_output()
            print("Loading climate data...")

        try:
            province = self.province_selector.value
            location = self.location_selector.value

            self.progress_bar.value = 40

            if location == 'ALL':
                # Load all locations
                dfs = []
                for i, loc in enumerate(self.locations):
                    try:
                        csv_path = f"{self.base_folder_path}/{province}/{loc}_daily_data.csv"
                        df_temp = pd.read_csv(csv_path)
                        dfs.append(df_temp)
                        self.progress_bar.value = 40 + (i / len(self.locations)) * 40
                    except:
                        continue
                self.df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
            else:
                csv_file_path = f"{self.base_folder_path}/{province}/{location}_daily_data.csv"
                self.df = pd.read_csv(csv_file_path)

            self.progress_bar.value = 80

            if self.df.empty:
                raise ValueError("No data found")

            # Process data
            if 'Date/Time' in self.df.columns:
                self.df['Date/Time'] = pd.to_datetime(self.df['Date/Time'])

            if 'Year' not in self.df.columns:
                self.df['Year'] = self.df['Date/Time'].dt.year
            if 'Month' not in self.df.columns:
                self.df['Month'] = self.df['Date/Time'].dt.month

            self.progress_bar.value = 90

            # Update column options
            self.update_column_options(self.analysis_selector.value)
            self.analyze_button.disabled = False

            self.progress_bar.value = 100

            # Resize for simple summary display
            if self.show_details_toggle.value:
                self.resize_output_area('table')
            else:
                self.resize_output_area('simple')

            # Display summary
            with self.output_area:
                clear_output() # Clear the 'Loading climate data...' message
                print("Data loaded successfully!")
                print(f"Records: {len(self.df):,}")
                print(f"Period: {self.df['Date/Time'].min().date()} to {self.df['Date/Time'].max().date()}")
                print(f"Location: {location or 'All locations'}")
                print(f"Province: {province}")

                if self.show_details_toggle.value:
                    print(f"\nAvailable columns:")
                    for col in sorted(self.df.columns):
                        if any(x in col for x in ['Temp', 'Precip', 'Snow', 'Rain', 'Gust']):
                            print(f"  • {col}")

        except Exception as e:
            self.resize_output_area('simple')
            with self.output_area:
                clear_output() # Clear the 'Loading climate data...' message
                print(f"Error loading data: {e}")

        finally:
            self.progress_bar.layout.display = 'none'
            self.progress_bar.value = 0

    def run_data_update(self, button):
        """Handles the logic for updating either a single file or all files."""
        update_type = self.update_type_selector.value
        province = self.province_selector.value

        # Ensure output area is cleared and resized for loading state before running update
        self.resize_output_area('loading')
        with self.output_area:
             clear_output()

        if update_type == 'single':
            location = self.location_selector.value
            if location == 'ALL' or location is None:
                self.resize_output_area('simple')
                with self.output_area:
                    print("Please select a specific location to update.")
                return

            csv_file_path = f"{self.base_folder_path}/{province}/{location}_daily_data.csv"
            self.update_single_location(csv_file_path, location, province)

        elif update_type == 'all':
            base_dir = f"{self.base_folder_path}/{province}"
            self.update_all_locations(base_dir, province)


    def update_single_location(self, file_path, location_name, province):
        """Updates a single CSV file with the most recent data."""
        # Output is captured by the with statement in run_data_update
        with self.output_area: # Wrap calls that print to stdout
            print(f"Starting update for {location_name}, {province}...")
            try:
                update_csv_file(file_path) # Call the original function
                # After update, reload the data if it was the currently loaded location
                if self.location_selector.value == location_name and self.province_selector.value == province:
                     # Clear just the update messages before showing the load summary
                     # This is a bit tricky with clear_output, might need a separate output area
                     # For now, let's just print the load summary after the update messages
                     self.load_data(None) # Reload the data for the currently selected location
                     print("Reloaded data for the selected location.")


            except Exception as e:
                # The outer run_data_update catch will handle errors and resize
                print(f"Error updating data for {location_name}, {province}: {e}") # Print error within output area

    def update_all_locations(self, base_dir, province):
        """Updates all CSV files in a given province."""
        # Output is captured by the with statement in run_data_update
        with self.output_area: # Wrap calls that print to stdout
            print(f"Starting update for all locations in {province}...")

            if not os.path.exists(base_dir):
                # The outer run_data_update catch will handle errors and resize
                print(f"Error: Directory {base_dir} not found.")
                return

            all_csv_files = glob.glob(os.path.join(base_dir, "*.csv"))

            if not all_csv_files:
                # The outer run_data_update catch will handle errors and resize
                print(f"No CSV files found in {base_dir}.")
                return

            print(f"Found {len(all_csv_files)} CSV files to update.")

            for i, file_path in enumerate(all_csv_files, 1):
                location_name = os.path.basename(file_path).replace('_daily_data.csv', '')
                print(f"\nUpdating file {i}/{len(all_csv_files)}: {location_name}...")

                try:
                    update_csv_file(file_path) # Call the original function
                    print(f"Update for {location_name} completed.")
                except Exception as e:
                    # The outer run_data_update catch will handle errors and resize
                    print(f"Error updating {location_name}: {str(e)}")

            print(f"\nAll files in {province} have been updated.")
            # If 'ALL' was the selected location, reload the concatenated data
            if self.location_selector.value == 'ALL' and self.province_selector.value == province:
                 self.load_data(None)
                 print("Reloaded data for all locations.")


    def run_analysis(self, button):
        if self.df is None:
            self.resize_output_area('simple')
            with self.output_area:
                clear_output()
                print("Please load data first")
            return

        analysis = self.analysis_selector.value

        # Determine appropriate sizing for analysis type
        if analysis in ['wind_freq', 'list_threshold', 'seasonal_precip', 'winter_snow']:
            self.resize_output_area('table')
        elif analysis == 'plot_monthly':
            self.resize_output_area('plot')
        elif analysis == 'overall_max_min' and self.show_details_toggle.value:
             self.resize_output_area('table') # Expand for additional results
        else:
            self.resize_output_area('simple')


        with self.output_area:
            clear_output()
            print(f"Running analysis...")

            try:
                # Route to appropriate analysis method
                if analysis == 'last_occurrence':
                    self.analyze_last_occurrence()
                elif analysis == 'overall_max_min':
                    self.analyze_overall_max_min()
                elif analysis == 'max_min_year':
                    self.analyze_max_min_year()
                elif analysis == 'max_min_month':
                    self.analyze_max_min_month()
                elif analysis in ['last_gte', 'last_lte']:
                    self.analyze_last_comparison()
                elif analysis in ['count_gte', 'count_lte']:
                    self.analyze_count_comparison()
                elif analysis == 'wind_freq':
                    self.analyze_wind_frequencies()
                elif analysis == 'seasonal_precip':
                    self.analyze_seasonal_precipitation()
                elif analysis == 'plot_monthly':
                    self.plot_monthly_precipitation()
                elif analysis == 'list_threshold':
                    self.list_threshold_occurrences()
                elif analysis == 'winter_snow':
                    self.analyze_winter_snowfall()

            except Exception as e:
                clear_output() # Clear the "Running analysis..." message
                print(f"Error running analysis: {e}")

    def analyze_last_occurrence(self):
        value = self.value_input.value
        column = self.column_selector.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Get all occurrences and store them
        all_matches = lookup_data(self.df, column, 'all_occurrences', value, location)
        self.analysis_results = all_matches.sort_values(by='Date/Time', ascending=False).reset_index(drop=True).to_dict('records') # Store as list of dicts
        self.current_result_index = 0 # Start with the last occurrence

        self.display_current_result() # Display the first result

    def analyze_overall_max_min(self):
        """Find overall max/min and store all matching occurrences."""
        column = self.column_selector.value
        lookup_type = self.max_min_selector.value.lower()
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Filter data based on location if specified
        data = self.df.copy()
        if location is not None:
            data = data[data['Station Name'].str.lower() == location.lower()]

        if data.empty:
            self.analysis_results = [] # Store as empty list
            self.current_result_index = 0
            self.display_current_result()
            return

        # Convert the column to numeric, coercing errors to NaN
        data[column] = pd.to_numeric(data[column], errors='coerce')
        data = data.dropna(subset=[column]) # Remove rows with NaN in the target column

        if data.empty:
            self.analysis_results = [] # Store as empty list
            self.current_result_index = 0
            self.display_current_result()
            return

        # Find unique extreme values and get their occurrences
        if lookup_type == 'max':
            # Sort by the column descending to get max values first
            sorted_data = data.sort_values(by=column, ascending=False)
        else: # lookup_type == 'min'
            # Sort by the column ascending to get min values first
            sorted_data = data.sort_values(by=column, ascending=True)

        # Get the top 20 distinct values
        distinct_values = sorted_data[column].unique()[:20]

        # Store results as a list of dictionaries, where each dict contains the value and its occurrences
        structured_results = []
        for value in distinct_values:
            occurrences = sorted_data[sorted_data[column] == value].to_dict('records')
            structured_results.append({
                'value': value,
                'occurrences': occurrences
            })

        # Store the structured results in analysis_results (as a list of dicts)
        self.analysis_results = structured_results
        self.current_result_index = 0 # Start with the first distinct extreme value

        self.display_current_result() # Display the first result


    def analyze_max_min_year(self):
        """Find max/min in specific year and store all matching occurrences."""
        year = self.year_input.value
        column = self.column_selector.value
        lookup_type = self.max_min_selector.value.lower()
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Get all occurrences matching the max/min in the year
        all_matches = lookup_data(self.df, column, f'{lookup_type}_in_year', year=year, location=location)
        self.analysis_results = all_matches.sort_values(by='Date/Time').reset_index(drop=True).to_dict('records') # Store as list of dicts
        self.current_result_index = 0 # Start with the earliest occurrence

        self.display_current_result() # Display the first result


    def analyze_max_min_month(self):
        """Find max/min in specific month and store all matching occurrences."""
        month = self.month_selector.value
        column = self.column_selector.value
        lookup_type = self.max_min_selector.value.lower()
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Get all occurrences matching the max/min in the month
        all_matches = lookup_data(self.df, column, f'{lookup_type}_in_month', month=month, location=location)
        self.analysis_results = all_matches.sort_values(by='Date/Time').reset_index(drop=True).to_dict('records') # Store as list of dicts
        self.current_result_index = 0 # Start with the earliest occurrence

        self.display_current_result() # Display the first result


    def analyze_last_comparison(self):
        """Analyze last occurrence greater/less than value and store all matching occurrences."""
        value = self.value_input.value
        column = self.column_selector.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None
        analysis = self.analysis_selector.value

        lookup_type = 'all_greater_than_or_equal' if analysis == 'last_gte' else 'all_less_than_or_equal'
        operator = '>=' if analysis == 'last_gte' else '<='

        # Get all occurrences and store them
        all_matches = lookup_data(self.df, column, lookup_type, value, location)
        self.analysis_results = all_matches.sort_values(by='Date/Time', ascending=False).reset_index(drop=True).to_dict('records') # Store as list of dicts
        self.current_result_index = 0 # Start with the last occurrence

        self.display_current_result() # Display the first result

    def display_current_result(self):
        """Displays the details of the current result from analysis_results."""
        with self.output_area:
            clear_output()
            if not self.analysis_results: # Check if the list is empty
                print("No results found for this analysis.")
                self.result_counter_label.value = "Result 0/0"
                self.update_navigation_buttons()
                return

            total_results = len(self.analysis_results)
            current_index = self.current_result_index

            analysis = self.analysis_selector.value
            column = self.column_selector.value
            location = self.location_selector.value if self.location_selector.value != 'ALL' else None

            # Update result counter label
            self.result_counter_label.value = f"Result {current_index + 1}/{total_results}"

            # Display header based on analysis type
            if analysis in ['last_occurrence', 'last_gte', 'last_lte', 'max_min_year', 'max_min_month']:
                 current_row = self.analysis_results[current_index] # Access as list of dicts
                 date = pd.to_datetime(current_row['Date/Time']).strftime('%Y-%m-%d')
                 value = current_row[column]
                 station = current_row['Station Name'] if 'Station Name' in current_row else 'N/A'

                 if analysis == 'last_occurrence':
                      search_value = self.value_input.value
                      print(f"Occurrence Analysis (Value = {search_value})")
                      print(f"Looking for: {column} = {search_value}")
                 elif analysis in ['last_gte', 'last_lte']:
                      search_value = self.value_input.value
                      operator = '>=' if analysis == 'last_gte' else '<='
                      print(f"Occurrence Analysis ({operator} {search_value})")
                      print(f"Looking for: {column} {operator} {search_value}")
                 elif analysis in ['max_min_year', 'max_min_month']:
                      lookup_type = self.max_min_selector.value.lower()
                      year = self.year_input.value if analysis == 'max_min_year' else 'N/A'
                      month = calendar.month_name[self.month_selector.value] if analysis == 'max_min_month' else 'N/A'
                      print(f"{lookup_type.capitalize()} Occurrence Analysis")
                      print(f"Column: {column}")
                      if analysis == 'max_min_year': print(f"Year: {year}")
                      if analysis == 'max_min_month': print(f"Month: {month}")

                 print(f"Location: {location or 'All locations'}")
                 print("-" * 40)

                 print(f"Date: {date}")
                 print(f"Value: {value}")
                 if not location: # Only show station name if not filtered by a specific location
                      print(f"Station: {station}")


            elif analysis == 'overall_max_min':
                 lookup_type = self.max_min_selector.value.lower()
                 print(f"Overall {lookup_type.capitalize()} Analysis")
                 print(f"Column: {column}")
                 print(f"Location: {location or 'All locations'}")
                 print("-" * 40)

                 # Display details for the current distinct extreme value
                 current_extreme_data = self.analysis_results[current_index]
                 extreme_value = current_extreme_data['value']
                 occurrences = current_extreme_data['occurrences']

                 print(f"Value: {extreme_value}")
                 print(f"Occurrences ({len(occurrences)}):")

                 # Display occurrences in a table
                 table_data = []
                 for occ in occurrences:
                     date = pd.to_datetime(occ['Date/Time']).strftime('%Y-%m-%d')
                     if location:
                         table_data.append([date])
                     else:
                         table_data.append([date, occ['Station Name']])

                 headers = ["Date"] if location else ["Date", "Station"]
                 print(tabulate(table_data, headers=headers, tablefmt="plain")) # Use plain format for cleaner list


            # Update navigation button states
            self.update_navigation_buttons()


    def display_previous_result(self, button):
        """Displays the previous result if available."""
        if self.current_result_index > 0:
            self.current_result_index -= 1
            self.display_current_result()

    def display_next_result(self, button):
        """Displays the next result if available."""
        if self.current_result_index < len(self.analysis_results) - 1:
            self.current_result_index += 1
            self.display_current_result()

    def update_navigation_buttons(self):
        """Enables/disables navigation buttons based on current result index."""
        total_results = len(self.analysis_results)
        self.prev_result_button.disabled = (self.current_result_index <= 0) or (total_results <= 1)
        self.next_result_button.disabled = (self.current_result_index >= total_results - 1) or (total_results <= 1)
        self.result_counter_label.value = f"Result {self.current_result_index + 1}/{total_results}" if total_results > 0 else "Result 0/0"


    def analyze_wind_frequencies(self):
        """Analyze wind gust frequencies with table sizing"""
        start_year = self.start_year_input.value
        end_year = self.end_year_input.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None
        gust_column = "Spd of Max Gust (km/h)"

        # Filter data
        filtered_df = self.df.copy()
        if start_year > 0:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year >= start_year]
        if end_year > 0:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year <= end_year]
        if location:
            filtered_df = filtered_df[filtered_df['Station Name'] == location]

        # Convert to numeric
        filtered_df[gust_column] = pd.to_numeric(filtered_df[gust_column], errors='coerce')
        filtered_df = filtered_df.dropna(subset=[gust_column])

        # This method is already wrapped in with self.output_area: in run_analysis
        clear_output()
        print(f"Wind Gust Frequency Analysis")
        print(f"Period: {start_year}-{end_year}")
        print(f"Location: {location or 'All locations'}")
        print(f"Total records: {len(filtered_df)}")
        print("-" * 50)

        # Calculate frequencies for different thresholds
        thresholds = [80, 90, 100, 110, 120, 130, 140, 150]
        total_years = end_year - start_year + 1 if start_year > 0 and end_year > 0 else 1 # Handle case with no year range

        table_data = []
        for threshold in thresholds:
            count = len(filtered_df[filtered_df[gust_column] >= threshold])
            if count > 0:
                if total_years > 0: # Avoid division by zero
                    freq_per_year = count / total_years

                    # Format frequency
                    if freq_per_year >= 1:
                        frequency_str = f"{freq_per_year:.1f} per year"
                    elif freq_per_year * 10 >= 1:
                        frequency_str = f"{freq_per_year * 10:.1f} per decade"
                    else:
                        frequency_str = f"{freq_per_year * 100:.1f} per century"
                else:
                     frequency_str = "N/A (No years in range)"


                table_data.append([f">={threshold} km/h", count, frequency_str])

        if table_data:
            print(tabulate(table_data, headers=["Threshold", "Count", "Frequency"], tablefmt="grid"))

        # Top 5 gusts
        top_5 = filtered_df.nlargest(5, gust_column)
        print(f"\nTop 5 Wind Gusts:")
        for i, (_, row) in enumerate(top_5.iterrows(), 1):
            date = row['Date/Time'].strftime('%Y-%m-%d')
            print(f"  {i}. {row[gust_column]:.1f} km/h on {date}")


    def find_next_extremes(self, column, lookup_type, location, current_value, current_date, num_additional=4):
        """Find the next highest/lowest values"""
        found_dates = {pd.to_datetime(current_date).date()}

        # This method is called within analyze_overall_max_min, which is wrapped
        for i in range(1, num_additional + 1):
            df_filtered = self.df.copy()
            df_filtered[column] = pd.to_numeric(df_filtered[column], errors='coerce')

            # Filter out previously found dates
            mask = ~df_filtered['Date/Time'].dt.date.isin(found_dates)
            if location:
                mask &= (df_filtered['Station Name'] == location)

            filtered_df = df_filtered[mask]

            if filtered_df.empty:
                break

            if lookup_type == 'max':
                next_idx = filtered_df[column].idxmax()
            else:
                next_idx = filtered_df[column].idxmin()

            next_row = filtered_df.loc[next_idx]
            next_value = next_row[column]
            next_date = next_row['Date/Time'].date()

            print(f"#{i+1}: {next_value} on {next_date}")
            found_dates.add(next_date)


    def analyze_count_comparison(self):
        """Count occurrences greater/less than value with checkbox-controlled year selection"""
        value = self.value_input.value
        column = self.column_selector.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None
        analysis = self.analysis_selector.value

        # Get year parameters based on checkbox
        use_range = self.use_year_range_toggle.value

        lookup_type = 'count_greater_than_or_equal' if analysis == 'count_gte' else 'count_less_than_or_equal'
        operator = '>=' if analysis == 'count_gte' else '<='

        # Filter data based on year selection
        filtered_df = self.df.copy()
        if location:
            filtered_df = filtered_df[filtered_df['Station Name'] == location]

        if use_range:
            start_year = self.start_year_input.value
            end_year = self.end_year_input.value
            filtered_df = filtered_df[
                (filtered_df['Date/Time'].dt.year >= start_year) &
                (filtered_df['Date/Time'].dt.year <= end_year)
            ]
            year_description = f"Period: {start_year}-{end_year}"
        else:
            single_year = self.year_input.value
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year == single_year]
            year_description = f"Year: {single_year}"

        # Convert to numeric and apply threshold
        filtered_df[column] = pd.to_numeric(filtered_df[column], errors='coerce')

        if analysis == 'count_gte':
            result_df = filtered_df[filtered_df[column] >= value]
        else:
            result_df = filtered_df[filtered_df[column] <= value]

        result_count = len(result_df)
        total_records = len(filtered_df[filtered_df[column].notna()])

        # This method is already wrapped in with self.output_area: in run_analysis
        clear_output()
        print(f"Count {operator} Analysis")
        print(f"Counting: {column} {operator} {value}")
        print(f"Location: {location or 'All locations'}")
        print(f"{year_description}")
        print("-" * 50)
        print(f"Total occurrences: {result_count}")
        print(f"Total records: {total_records}")
        if total_records > 0:
            percentage = (result_count / total_records) * 100
            print(f"Percentage: {percentage:.1f}%")

        # Show additional details if enabled
        if self.show_details_toggle.value and result_count > 0:
            print(f"\nAdditional Details:")

            # Show yearly breakdown for ranges
            if use_range:
                yearly_counts = result_df.groupby(result_df['Date/Time'].dt.year).size()
                print(f"\nYearly breakdown:")
                for year, count in yearly_counts.items():
                    year_total = len(filtered_df[
                        (filtered_df['Date/Time'].dt.year == year) &
                        (filtered_df[column].notna())
                    ])
                    if year_total > 0:
                        year_pct = (count / year_total) * 100
                        print(f"  {year}: {count} occurrences ({year_pct:.1f}%)")

            # Show monthly breakdown for single year
            else:
                monthly_counts = result_df.groupby(result_df['Date/Time'].dt.month).size()
                single_year = self.year_input.value
                print(f"\nMonthly breakdown for {single_year}:")
                for month, count in monthly_counts.items():
                    month_total = len(filtered_df[
                        (filtered_df['Date/Time'].dt.month == month) &
                        (filtered_df[column].notna())
                    ])
                    if month_total > 0:
                        month_pct = (count / month_total) * 100
                        print(f"  {calendar.month_name[month]}: {count} occurrences ({month_pct:.1f}%)")

    def analyze_seasonal_precipitation(self):
        """Analyze seasonal precipitation with table sizing"""
        year = self.year_input.value
        column = self.column_selector.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Filter data for the year
        year_df = self.df[
            ((self.df['Date/Time'].dt.year == year) |
            ((self.df['Date/Time'].dt.year == year - 1) & (self.df['Date/Time'].dt.month == 12)))
        ].copy()

        if location:
            year_df = year_df[year_df['Station Name'] == location]

        # Convert to numeric
        year_df[column] = pd.to_numeric(year_df[column], errors='coerce')

        # Add season column
        year_df['Season'] = year_df['Date/Time'].dt.month.apply(get_meteorological_season)

        # Calculate seasonal totals
        seasonal_totals = year_df.groupby('Season')[column].sum()

        # This method is already wrapped in with self.output_area: in run_analysis
        clear_output()
        print(f"Seasonal Precipitation Analysis")
        print(f"Year: {year}")
        print(f"Column: {column}")
        print(f"Location: {location or 'All locations'}")
        print("-" * 50)

        season_order = ['Winter', 'Spring', 'Summer', 'Fall']
        table_data = []

        for season in season_order:
            if season in seasonal_totals:
                total = seasonal_totals[season]
                table_data.append([season, f"{total:.1f}"])

        # Annual total
        annual_total = year_df[year_df['Date/Time'].dt.year == year][column].sum()
        table_data.append(["Annual", f"{annual_total:.1f}"])

        print(tabulate(table_data, headers=["Season", "Total"], tablefmt="grid"))
        print("\nNote: Winter includes December of previous year")

    def list_threshold_occurrences(self):
        """List all occurrences above threshold with table sizing"""
        threshold = self.value_input.value
        column = self.column_selector.value
        start_year = self.start_year_input.value
        end_year = self.end_year_input.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Filter data
        filtered_df = self.df.copy()
        if start_year > 0:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year >= start_year]
        if end_year > 0:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year <= end_year]
        if location:
            filtered_df = filtered_df[filtered_df['Station Name'] == location]

        # Convert to numeric and filter by threshold
        filtered_df[column] = pd.to_numeric(filtered_df[column], errors='coerce')
        threshold_df = filtered_df[filtered_df[column] >= threshold].sort_values('Date/Time')

        # This method is already wrapped in with self.output_area: in run_analysis
        clear_output()
        print(f"Threshold Occurrences Analysis")
        print(f"Threshold: {column} >= {threshold}")
        print(f"Period: {start_year}-{end_year}")
        print(f"Location: {location or 'All locations'}")
        print("-" * 50)

        if threshold_df.empty:
            print(f"No occurrences found >= {threshold}")
        else:
            print(f"Total occurrences: {len(threshold_df)}")
            print(f"\nAll occurrences:")

            table_data = []
            display_limit = 50 if self.show_details_toggle.value else 20

            for i, (_, row) in enumerate(threshold_df.iterrows()):
                if i >= display_limit:
                    break
                date = row['Date/Time'].strftime('%Y-%m-%d')
                value = row[column]
                if location:
                    table_data.append([date, f"{value:.1f}"])
                else:
                    table_data.append([date, f"{value:.1f}", row['Station Name']])

            headers = ["Date", "Value"] if location else ["Date", "Value", "Station"]
            print(tabulate(table_data, headers=headers, tablefmt="grid"))

            if len(threshold_df) > display_limit:
                remaining = len(threshold_df) - display_limit
                print(f"\n... and {remaining} more occurrences")
                if not self.show_details_toggle.value:
                    print("(Enable 'Show details' to see up to 50 results)")

    def analyze_winter_snowfall(self):
        """Analyze top winter snowfall days with table sizing"""
        winter_start_year = self.year_input.value
        winter_end_year = winter_start_year + 1
        snow_column = "Total Snow (cm)"
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Filter for winter period (Nov 1 to Apr 30)
        winter_mask = (
            ((self.df['Date/Time'].dt.year == winter_start_year) &
            (self.df['Date/Time'].dt.month >= 11)) |
            ((self.df['Date/Time'].dt.year == winter_end_year) &
            (self.df['Date/Time'].dt.month <= 4))
        )

        if location:
            winter_mask &= (self.df['Station Name'] == location)

        winter_df = self.df[winter_mask].copy()
        winter_df[snow_column] = pd.to_numeric(winter_df[snow_column], errors='coerce')
        winter_df = winter_df.dropna(subset=[snow_column])

        # Get top 5 snowfall days
        top_5_snow = winter_df.sort_values(by=snow_column, ascending=False).head(5)

        # This method is already wrapped in with self.output_area: in run_analysis
        clear_output()
        print(f"Winter Snowfall Analysis")
        print(f"Winter: {winter_start_year}-{winter_end_year}")
        print(f"Location: {location or 'All locations'}")
        print("-" * 50)

        if top_5_snow.empty:
            print(f"No snow data found for winter {winter_start_year}-{winter_end_year}")
        else:
            print(f"Top 5 Snowfall Days:")

            table_data = []
            for i, (_, row) in enumerate(top_5_snow.iterrows(), 1):
                date = row['Date/Time'].strftime('%Y-%m-%d')
                snow = row[snow_column]
                if location:
                    table_data.append([i, date, f"{snow:.1f}"])
                else:
                    table_data.append([i, date, f"{snow:.1f}", row['Station Name']])

            headers = ["Rank", "Date", "Snow (cm)"] if location else ["Rank", "Date", "Snow (cm)", "Station"]
            print(tabulate(table_data, headers=headers, tablefmt="grid"))

            # Statistics
            total_snow = winter_df[snow_column].sum()
            days_with_snow = len(winter_df[winter_df[snow_column] > 0])
            avg_snow_day = winter_df[winter_df[snow_column] > 0][snow_column].mean()

            print(f"\nWinter Statistics:")
            stats_data = [
                ["Total snowfall", f"{total_snow:.1f} cm"],
                ["Days with snow", f"{days_with_snow}"],
            ]
            if days_with_snow > 0:
                stats_data.append(["Average per snow day", f"{avg_snow_day:.1f} cm"])
            else:
                stats_data.append(["Average per snow day", "N/A"])


            print(tabulate(stats_data, headers=["Statistic", "Value"], tablefmt="grid"))

    def plot_monthly_precipitation(self):
        """Plot monthly precipitation."""
        start_year = self.start_year_input.value
        end_year = self.end_year_input.value
        month = self.month_selector.value
        precip_type = self.precip_type_selector.value
        location = self.location_selector.value if self.location_selector.value != 'ALL' else None

        # Determine which columns to use based on precip_type
        precip_cols = []
        if precip_type == 'rain':
            precip_cols = [col for col in self.df.columns if 'Rain' in col]
        elif precip_type == 'snow':
            precip_cols = [col for col in self.df.columns if 'Snow' in col]
        elif precip_type == 'precip':
            precip_cols = [col for col in self.df.columns if 'Precip' in col]
        elif precip_type == 'rain_snow':
            precip_cols = [col for col in self.df.columns if 'Rain' in col or 'Snow' in col]

        if not precip_cols:
            with self.output_area:
                clear_output()
                print(f"No relevant precipitation columns found for type: {precip_type}")
            return

        # Filter data
        filtered_df = self.df.copy()
        if start_year > 0:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year >= start_year]
        if end_year > 0:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.year <= end_year]
        if month:
            filtered_df = filtered_df[filtered_df['Date/Time'].dt.month == month]
        if location:
            filtered_df = filtered_df[filtered_df['Station Name'] == location]

        if filtered_df.empty:
            with self.output_area:
                clear_output()
                print("No data found for the selected criteria.")
            return

        # Calculate daily total for selected precipitation types
        filtered_df['Daily Total Precip'] = filtered_df[precip_cols].sum(axis=1, numeric_only=True)

        # Group by year and month and sum the daily totals
        monthly_precip = filtered_df.groupby(filtered_df['Date/Time'].dt.to_period('M'))['Daily Total Precip'].sum()
        monthly_precip.index = monthly_precip.index.to_timestamp() # Convert PeriodIndex to DatetimeIndex for plotting

        with self.output_area:
            clear_output()
            plt.figure(figsize=(12, 6))
            monthly_precip.plot(kind='bar' if len(monthly_precip) <= 24 else 'line') # Use bar for <= 2 years, line otherwise
            plt.title(f"Monthly {precip_type.capitalize()} Precipitation ({calendar.month_name[month] if month else 'All Months'})")
            plt.xlabel("Month")
            plt.ylabel(f"Total Precipitation (mm)")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()


    def clear_output(self, button):
        with self.output_area:
            clear_output()
        # Reset analysis results and navigation state
        self.analysis_results = [] # Reset to empty list
        self.current_result_index = 0
        self.update_navigation_buttons()

    def setup_layout(self):
        # Header with theme-adaptive styling
        header = widgets.HTML(
            "<div style='text-align: left; padding: 15px; margin-bottom: 10px;'>"
            "<h2>🌦️ Interactive Weather Data Analyzer</h2>"
            "<p>Advanced climate analysis with interactive widgets</p>"
            "</div>"
        )

        # Selection panel with options included
        selection_panel = widgets.VBox([
            widgets.HTML("<h3>📍 Data Selection & Update</h3>"), # Updated title
            widgets.HBox([self.province_selector, self.location_selector]),
            widgets.HBox([self.show_details_toggle, self.auto_refresh_toggle]),
            widgets.HBox([self.load_button, self.progress_bar]),
            widgets.VBox([ # New VBox for update options
                widgets.HTML("<h4>Update Options:</h4>"),
                self.update_type_selector,
                self.run_update_button
            ], layout=widgets.Layout(border='1px dashed var(--jp-border-color2, #ccc)', padding='10px', margin='10px 0px')) # Added dashed border


        ], layout=widgets.Layout(
            border='1px solid var(--jp-border-color1, #ddd)',
            padding='10px',
            margin='5px'
        ))

        # Analysis panel with year range checkbox
        analysis_panel = widgets.VBox([
            widgets.HTML("<h3>Analysis Configuration</h3>"),
            self.analysis_selector,
            self.column_selector,
            widgets.HBox([self.value_input, self.year_input, self.month_selector]),
            widgets.HBox([self.start_year_input, self.end_year_input, self.use_year_range_toggle]),  # Added checkbox here
            widgets.HBox([self.max_min_selector, self.precip_type_selector]),
            widgets.HBox([self.analyze_button, self.clear_button]),
        ], layout=widgets.Layout(border='1px solid var(--jp-border-color1, #ddd)', padding='10px', margin='5px', width='auto'))

        # Combine all panels vertically
        self.widget = widgets.VBox([
            header,
            widgets.HBox([selection_panel, analysis_panel]),
            self.results_plots_panel # Use the results_plots_panel defined in setup_widgets
        ])

    def display(self):
        display(self.widget)
        # Auto-initialize province and analysis type
        self.on_province_change({'new': self.province_selector.value})
        # Trigger analysis change to set up default state
        self.on_analysis_change({'new': self.analysis_selector.value})

# Widget Display

In [ ]:
# Initialize and display
analyzer = InteractiveWeatherAnalyzer('/content/drive/MyDrive/Climate Data/Daily Data')
analyzer.display()